<a href="https://colab.research.google.com/github/realBagher/DeepLearning-Specializattion-/blob/main/Named-Entity-Recognition/Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
from transformers import pipeline 

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [5]:
!nvcc --version

/bin/bash: nvcc: command not found


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [8]:
import torch 

In [9]:
torch.cuda.is_available()

True

In [10]:
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

In [11]:
torch.cuda.current_device()

0

In [12]:
ner = pipeline('ner',aggregation_strategy='simple',device=0)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [13]:
type(ner)

transformers.pipelines.token_classification.TokenClassificationPipeline

In [14]:
import pickle 

In [ ]:
!wget https://www.lazyprogrammer.me/course_files/nlp/ner_train.pkl
!wget https://www.lazyprogrammer.me/course_files/nlp/ner_test.pkl

In [16]:
with open('ner_train.pkl','rb' ) as f:
  corpus_train = pickle.load(f)

with open('ner_test.pkl','rb' ) as f:
  corpus_test = pickle.load(f)
   

In [ ]:
corpus_test

In [18]:
inputs = []
targets = []

In [19]:
for sentence_tag_pairs in corpus_test:
  tokens =[]
  target = []
  for token,tag in sentence_tag_pairs:
    tokens.append(token)
    target.append(tag)
  inputs.append(tokens)
  targets.append(target)

In [20]:
inputs[9]

['He',
 'was',
 'well',
 'backed',
 'by',
 'England',
 'hopeful',
 'Mark',
 'Butcher',
 'who',
 'made',
 '70',
 'as',
 'Surrey',
 'closed',
 'on',
 '429',
 'for',
 'seven',
 ',',
 'a',
 'lead',
 'of',
 '234',
 '.']

In [21]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()

In [22]:
ner(detokenizer.detokenize(inputs[9]))

[{'end': 29,
  'entity_group': 'LOC',
  'score': 0.99967515,
  'start': 22,
  'word': 'England'},
 {'end': 50,
  'entity_group': 'PER',
  'score': 0.99974275,
  'start': 38,
  'word': 'Mark Butcher'},
 {'end': 72,
  'entity_group': 'ORG',
  'score': 0.9996264,
  'start': 66,
  'word': 'Surrey'}]

In [23]:
detokenizer.detokenize(inputs[9])

'He was well backed by England hopeful Mark Butcher who made 70 as Surrey closed on 429 for seven, a lead of 234.'

In [24]:
targets[9]

['O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [25]:
def compute_prediction(tokens,input_,ner_results):

  predicted_tags = []
  state = 'O'
  current_index = 0
  for token in tokens:
    index = input_.find(token)
    assert(index >= 0 )
    current_index += index

    tag = 'O'
    for entity in ner_result:
      if current_index >= entity['start'] and current_index < entity['end']:
        if state =='O':
          state = 'B'
        else:
          state = 'I'
        tag = f"{state} - {entity['entity_group']}"
        break
    if tag == 'O':
      state= 'O'
    predicted_tags.append(tag)

    input_ = input_[index + len(token):]
    current_index += len(token)


  assert(len(predicted_tags) == len(tokens))
  return predicted_tags


In [26]:
input_ = detokenizer.detokenize(inputs[9])
ner_result = ner(input_)
ptags = compute_prediction(inputs[9],input_,ner_result)

In [27]:
from sklearn.metrics import accuracy_score , f1_score

In [28]:
accuracy_score(targets[9],ptags)

0.84

In [29]:
for targ,pred in zip(targets[9],ptags):
  print(targ,pred)

O O
O O
O O
O O
O O
B-LOC B - LOC
O O
B-PER B - PER
I-PER I - PER
O O
O O
O O
O O
B-ORG B - ORG
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O
O O


In [30]:
detok_inputs = []
for tokens in inputs:
  text = detokenizer.detokenize(tokens)
  detok_inputs.append(text)

In [31]:
ner_results = ner(detok_inputs)

In [32]:
predictions = []
for tokens, text, ner_result in zip(inputs, detok_inputs, ner_results):
  pred = compute_prediction(tokens, text, ner_result)
  predictions.append(pred)
  

In [33]:
detok_inputs = []
for tokens in inputs:
  text = detokenizer.detokenize(tokens)
  detok_inputs.append(text)

In [34]:
ner_results = ner(detok_inputs)

In [36]:
predictions = []
for tokens, text, ner_result in zip(inputs, detok_inputs, ner_results):
  pred = compute_prediction(tokens,text,ner_result)
  predictions.append(pred)

In [37]:
def flatten(list_of_lists):
  flattened = [val for sublist in list_of_lists for val in sublist]
  return flattened

In [43]:
flat_predictions = flatten(predictions)
flat_targets = flatten(targets)
assert(len(flat_predictions)==len(flat_targets))

In [44]:
accuracy_score(flat_targets,flat_predictions)


0.8338941692740618

In [45]:
f1_score(flat_targets,flat_predictions,average='macro')

0.05870784824015689